In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<IPython.core.display.Javascript object>

## Arranging Libraries

In [8]:
! pip install nb_black   ## Code Beautifier


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


<IPython.core.display.Javascript object>

In [3]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [4]:
pd.set_option('max_columns',None)  ### To see each column on data

OptionError: 'Pattern matched multiple keys'

<IPython.core.display.Javascript object>

In [5]:
import seaborn as sns 
import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

## Read DATA

In [6]:
df = pd.read_csv('/kaggle/input/stajveriler/stajverilertemiz/satislar_csv.csv') ### Read data set

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/stajveriler/stajverilertemiz/satislar_csv.csv'

<IPython.core.display.Javascript object>

In [ ]:
fig,ax = plt.subplots(figsize=(20,9))
ax = sns.heatmap(df.corr(),annot=True)

In [ ]:
df.columns = df.columns.str.replace('\s+', '') ### The spaces between columns removed

## Getting Rid Off from Canceled Orders

In [ ]:
df_canceled = pd.read_excel('/kaggle/input/canceled/canceled.xlsx') #### We have dropped Cancelled datas

In [ ]:
df.shape

In [ ]:
df_canceled.head()

In [ ]:
df = df[~df['NORDERID'].isin(df_canceled['nOrderId'])]

In [ ]:
df.shape

## Converting 'DRECEIEVEDDATE' to datetime

In [ ]:
df['DRECEIEVEDDATE'] = pd.to_datetime(df['DRECEIEVEDDATE']) ### we convert the column to datetime

In [ ]:
df['DRECEIEVEDDATE'] = df['DRECEIEVEDDATE'].dt.strftime('%d/%m/%Y') ### The column must be seen in this form to convert currencies in further steps.

In [ ]:
df['DRECEIEVEDDATE'] = pd.to_datetime(df['DRECEIEVEDDATE'])

In [ ]:
df.head()

## Removing UNKNOWN country and Currency and PLN currency

In [ ]:
df.columns

In [ ]:
df = df[(df['COUNTRY'] != "UNKNOWN") | (df["CURRENCY"] != "UNK")] 
## We don't have any idea about the currency and country and also we don't know the revenue from this orders.
## 320 columns are removed.

In [ ]:
df.shape

In [ ]:
df = df[df['CURRENCY'] != 'PLN'] ## We dropped that because we have only one row with PLN----TOTAL = 307 PLN

In [ ]:
df.shape

## Filling Nulls on Customer ID and CPostCode

In [ ]:
df['CUSTOMER_ID'].fillna('UNKNOWN',inplace=True) ## 21 Null values of CUSTOMER_ID are filled with 'UNKNOWN'.

In [ ]:
df[df['CUSTOMER_ID']== 'UNKNOWN']

In [ ]:
df['CPOSTCODE'].fillna('UNKNOWN',inplace=True) ## Null Values on CPOSTCODE filled by 'UNKNOWN'. Becuase we didn't want to lose the data.

In [ ]:
df.shape

In [ ]:
df.shape

## Filling NULL values on Processdate

In [ ]:
df.isnull().sum()

In [ ]:
df[df['PROCESSEDDATE'].isna()] ## Processdateler sorulacak. Iadeler tablosunda genel olarak bunlardan var.

In [ ]:
df[df['NORDERID'] == 105333 ] ### We see the same values on the iadeler may be we should drop null process dates.

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.head()

## Dropping Tracking Number Column and Dispatchunitcost,TotalWeight

In [ ]:
## This are also uncessary columns full of nulls and by 'TRACKINGNUMBER' lots of '2E+1' etc.)
df.drop('DISPATCHSTOCKUNITCOST',axis=1,inplace=True)
df.drop('TRACKINGNUMBER',axis=1,inplace=True)
df.drop('TOTALWEIGHT',axis=1,inplace=True)

In [ ]:
df.shape

In [ ]:
df.head()

## Dealing with UNK currencies

In [ ]:
df4 = df[df['CURRENCY'] == 'UNK']  ## Exhange Library can not convert the UNK currencies.

In [ ]:
df = df[df['CURRENCY']!= 'UNK']  ###  We don't know Country and same as currency

## Exchange Rates Changing

In [ ]:
!pip install forex_python   ### This is the library we use to exchange currencies.

In [ ]:
from forex_python.converter import CurrencyRates ### The library we will use to convert currencies to GBP.

In [ ]:
from datetime import datetime                 ### The library that formats our 'DRECEIVEDATE' column.

In [ ]:
c = CurrencyRates()                         ### Function which converts our currencies

In [ ]:
df.dtypes

In [ ]:
df['DRECEIEVEDDATE'] = df['DRECEIEVEDDATE'].dt.strftime('%d.%m.%Y') ### Date Column must be in this give format.

In [ ]:
df['TOTAL'] = df.apply( lambda x: c.convert(x.CURRENCY, 'GBP', 
                                             float(str(x.TOTAL).replace(',', '.')), 
                                             datetime.strptime(x.DRECEIEVEDDATE, "%d.%m.%Y").date()), axis = 1)

## Here we converted values on the'TOTAL' from diffrent currencies to GBP

In [ ]:
df.head()

In [ ]:
df['SUBTOTAL'] = df.apply( lambda x: c.convert(x.CURRENCY, 'GBP', 
                                             float(str(x.SUBTOTAL).replace(',', '.')), 
                                             datetime.strptime(x.DRECEIEVEDDATE, "%d.%m.%Y").date()), axis = 1)

## Here we converted values on the'SUBTOTAL' from diffrent currencies to GBP

In [ ]:
df['PURCHASEPRICE'] = df.apply( lambda x: c.convert(x.CURRENCY, 'GBP', 
                                             float(str(x.PURCHASEPRICE).replace(',', '.')), 
                                             datetime.strptime(x.DRECEIEVEDDATE, "%d.%m.%Y").date()), axis = 1)

## Here we converted values on the'SUBTOTAL' from diffrent currencies to GBP

In [ ]:
df[df['CURRENCY'] == 'EUR']

In [ ]:
df['CURRENCY'] = df['CURRENCY'].replace(['EUR'],'GBP')

In [ ]:
df['CURRENCY'] = df['CURRENCY'].replace(['SEK'],'GBP')

In [ ]:
df[df['CURRENCY'] != 'GBP'] ## We proved all currencies successfully converted to GBP.

In [ ]:
df4.dtypes

In [ ]:
df4.head(25)

In [ ]:
df.dtypes

In [ ]:
df4.head()

In [ ]:
df['DRECEIEVEDDATE'] = pd.to_datetime(df['DRECEIEVEDDATE']) 

## We had to convert it again to datetime otherwise we couldn't get to step of concatenation

In [ ]:
df = pd.concat([df,df4],axis=0) ### We have concatenated our df with df4 to gather UNK currencies with GPB's.

In [ ]:
df.shape

In [ ]:
df[df['CURRENCY'] == 'GBP']

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
# df['TOTAL'] = df['TOTAL'].apply(lambda x: '%.2f' % x)
# df['SUBTOTAL'] = df['SUBTOTAL'].apply(lambda x: '%.2f' % x)
# df['TAX'] = df['TAX'].apply(lambda x: '%.2f' % x)
# df['PURCHASEPRICE'] = df['PURCHASEPRICE'].apply(lambda x: '%.2f' % x)

In [ ]:
df.head()

### 1.Adding Features

## Adding Quantity Price with tax and without tax

In [ ]:
df.dtypes

In [ ]:
df['TP_per_Quantity'] = df['TOTAL'].astype('float') / df['ORDERITEMQUANTITY'].astype(float)  
## With Tax (The Income per Quantity with tax.)

In [ ]:
df.dtypes

In [ ]:
df['ST_per_Quantity'] = df['SUBTOTAL'].astype('float') / df['ORDERITEMQUANTITY'].astype('float') 
## With Out Tax(The Income without tax per quantity  )

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

## Adding Inflation Rates Turkey and UK

In [ ]:
df5 = pd.read_excel('/kaggle/input/inflation/Inflation.xlsx') ### We have created a new data set for Iflation

In [ ]:
df.rename(columns = {'DRECEIEVEDDATE_YEAR' : 'YEAR'},inplace=True)

In [ ]:
df = df.merge(df5, left_on='YEAR', right_on='YEAR', how='left') ## Yearly Inflation of UK for the 2022 as a guess

In [ ]:
df.head()

In [ ]:
df.shape

## Adding Net_Profit

In [ ]:
df['Net_Profit'] = df['SUBTOTAL'].astype('float') - df['PURCHASEPRICE'].astype('float') 

## We would like to know net income for each sale.

In [ ]:
df.head()

In [ ]:
df.head() ## CPI is for INFLATION

In [ ]:
df.isnull().sum()

In [ ]:
df.rename(columns = {'YEAR' : 'DRECEIEVEDDATE_YEAR'},inplace = True)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

##  Adding DayTime Feature

In [ ]:
def daytime_finder(x):
    if x>=0 and x<=6:
        return'Early Morning'                          
    if x>6 and x<=12:
        return 'Morning'
    if x>12 and x<=18:
        return 'AfterNoon'
    elif x>18 and x<=23:
        return 'Night'  
    
    #### We have splitted up a day to 4 parts on 6 hours base for each part.

In [ ]:
df['DayTime'] = df.DRECEIEVEDDATE_HOUR.apply(lambda x:daytime_finder(x))

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
df.dtypes

## Copying DF to keep null data

In [ ]:
#### Burada df e copy cekilip null veriler ile kaydedilecek kaydedilecek. (df must be copied here and we will go further with another dataframe)

df_null = df.copy() ### To guarantee null processdates

In [ ]:
df_null.isnull().sum()

In [ ]:
 #df_null.to_excel('df_null_processdate.xlsx')

In [ ]:
df_float = df_null.copy() ##### To guarentee float data

In [ ]:
df_float = df_float[df_float['PROCESSEDDATE'].notnull()]

In [ ]:
df_float.shape

In [ ]:
df_float.isnull().sum()

In [ ]:
df_float.to_excel('satislar_update.xlsx')

In [ ]:
df_float.dtypes

In [ ]:
# df[['SUBTOTAL', 'TAX', 'TOTAL', 'PURCHASEPRICE', 'TP_per_Quantity', 
    'ST_per_Quantity', 'UK_CPI', 'Turkey_CPI', 'Net_Profit']] = df[['SUBTOTAL', 'TAX', 'TOTAL', 'PURCHASEPRICE', 'TP_per_Quantity', 
                                                                                                                                    'ST_per_Quantity', 'UK_CPI', 'Turkey_CPI', 'Net_Profit']].round(2)

In [ ]:
df.head()

## Dropping Extra Process_date features

## Processdate ---- 3144 Null Value

In [ ]:
## Uncessary Columns and nulls
df.drop(['PROCESSEDDATE_YEAR','PROCESSEDDATE_MONTH','PROCESSEDDATE_DAY',
         'PROCESSEDDATE_HOUR','PROCESSEDDATE_DAY_NAME','PROCESSEDDATE_D'],axis=1,inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.PROCESSEDDATE.fillna(df['DRECEIEVEDDATE'],inplace=True) ## Just not to lose 3144 datas

In [ ]:
df.head()

In [ ]:
df.to_excel('satislar(GPB)v2.xlsx') ### Null ---Processdate--- FILLED

## Converting Dates

In [ ]:
df['DRECEIEVEDDATE'] = pd.to_datetime(df['DRECEIEVEDDATE'])

## Making Visualization

In [ ]:
df.head()

#### 1) Finding the most attractive time for the customers

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#### 1.1 Most Ordered Years

In [ ]:
fig,ax = plt.subplots(figsize=(16,9))
ax = sns.countplot(data=df,x='DRECEIEVEDDATE_YEAR',order = df['DRECEIEVEDDATE_YEAR'].value_counts().index)
ax.bar_label(ax.containers[0]);

#### 1.2 The Hours with most orders

In [ ]:
fig,ax = plt.subplots(figsize=(25,13))
sns.countplot(data=df,x='DRECEIEVEDDATE_HOUR')
sns.lineplot(data=df,x ='DRECEIEVEDDATE_HOUR')
ax.bar_label(ax.containers[0]);

#### 1.3 Days of the Week which is the most attractive days for the customers

In [ ]:
fig,ax = plt.subplots(figsize=(16,9))
sns.countplot(data=df,x='DRECEIEVEDDATE_DAY_NAME',order=df['DRECEIEVEDDATE_DAY_NAME'].value_counts().index)
ax.bar_label(ax.containers[0]);

#### 1.4 The most attractive Month of the year

In [ ]:
fig,ax = plt.subplots(figsize=(16,9))
sns.countplot(data=df,x='DRECEIEVEDDATE_MONTH')
ax.bar_label(ax.containers[0]);

#### 1.5 The most attractive dailytime for customers

In [ ]:
df['DayTime']=df.DayTime.astype('str')

In [ ]:
fig,ax = plt.subplots(figsize=(16,9))
sns.countplot(data=df,x='DayTime',order=df['DayTime'].value_counts().index)
ax.bar_label(ax.containers[0]);

#### 2) Finding the Bestseller Products

In [ ]:
fig,ax = plt.subplots(figsize=(16,9))
plt.xticks(rotation=45)
sns.countplot(data=df,x='ORDERITEMTITLE',order=df['ORDERITEMTITLE'].value_counts().iloc[:10].index)
ax.bar_label(ax.containers[0],rotation=45);